##### Imports:

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
# deep learning

import numpy as np
# linear algebra

from scipy.integrate import solve_ivp
# dataset generaion solver

from torch.utils.data import Dataset, DataLoader
from torch.utils.data import random_split
# data handling

#### Datset Generation (Synthetic But Correct as per SciPy)

- Sample generation functions

In [2]:
def generate_ode_params(n_samples=1000):
    # generate coeffs a,b,c and y_0, for dy/dx = ax + by + c, initial condition y_0.

    np.random.seed(42)
    
    params = []
    for _ in range(n_samples):
        a = np.random.uniform(-2, 2)
        b = np.random.uniform(-2, 2)
        c = np.random.uniform(-2, 2)
        y0 = np.random.uniform(-2, 2)

        # why (-2,2): numerical stability
        # otherwise, solutions to the diff eqns may vary wildly which will cause
        # problems in computing MSE loss.

        # eg. if params: (a=0, b=10, c=0, y_0=1) dy/dx = 0x + 10y + 0
        # solution: y = e^(10x), so as x -> infty, major changes

        # LIMITATION: less generalizable to larger magnitude solutions
        # neural net may not learn everything

        params.append((a, b, c, y0))
    return params


def solve_ode(a, b, c, y0, x_span=(0, 5), num_points=50):

    # solve dy/dx = ax + by + c with initial condition y_0 by enumeration

    # essentially, numerically solving and "plotting" the continuous curve at 50 points, 
    # in the interval (0,5)

    def ode_func(x, y):
        return a*x + b*y + c

    x_eval = np.linspace(x_span[0], x_span[1], num_points)
    # 50 points of a curve between 0 to 5

    sol = solve_ivp(ode_func, x_span, [y0], t_eval=x_eval, method='RK45')

    # sol.y[0] == solution for y at x_eval
    return x_eval, sol.y[0]


- Verifying with an online solver for some random cases.

In [3]:
synthetic_dataset_param_tuples = generate_ode_params(3)

for ai, bi, ci, y0i in synthetic_dataset_param_tuples:
    (x,y) = solve_ode(ai, bi, ci, y0i)
    print(f"Params: ai = {ai}, bi = {bi}, ci = {ci}, y0i = {y0i}")
    print(f"y(x) at x = 0 is {y[0]}, at x = 5 is {y[-1]}")
    print()


Params: ai = -0.50183952461055, bi = 1.8028572256396647, ci = 0.9279757672456204, y0i = 0.3946339367881464
y(x) at x = 0 is 0.3946339367881464, at x = 5 is 6204.165121464221

Params: ai = -1.375925438230254, bi = -1.3760219186551894, ci = -1.7676655513272022, y0i = 1.4647045830997407
y(x) at x = 0 is 1.4647045830997407, at x = 5 is -5.555346137621467

Params: ai = 0.4044600469728352, bi = 0.832290311184182, ci = -1.9176620228167902, y0i = 1.8796394086479773
y(x) at x = 0 is 1.8796394086479773, at x = 5 is 9.515691220750707



In [4]:
class ODEDataset_Linear(torch.utils.data.Dataset):
    def __init__(self, n_samples=1000, x_span=(0, 5), num_points=50):
        self.x_eval = np.linspace(x_span[0], x_span[1], num_points)
        self.samples = []
        
        params_list = generate_ode_params(n_samples)
        for a, b, c, y0 in params_list:
            _, y = solve_ode(a, b, c, y0, x_span, num_points)
            
            # store as (params, solution)
            self.samples.append( (np.array([a, b, c, y0], dtype=np.float32), y.astype(np.float32)) )
            # store the value of y for x = 5

    def __len__(self):
        return len(self.samples)
    
    def __getitem__(self, idx):
        params, y = self.samples[idx]
        return torch.from_numpy(params), torch.from_numpy(y)

In [5]:
full_dataset = ODEDataset_Linear(n_samples=1000)

# Split into 600 train + 100 validation + 300 Test (60/10/30 split)
train_dataset, val_dataset, test_dataset = random_split(full_dataset, [0.6, 0.1, 0.3], generator = torch.Generator().manual_seed(42))

# Create DataLoaders (batch size 32)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32)

#### Model Architecture:

##### Hypernetwork
Trains the weights for the solution network (up next).

In [6]:
class HyperNet(nn.Module):
    def __init__(self, param_dim=4, hidden_dim=64, output_dim=25):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(param_dim, hidden_dim),
            # hidden layer, 64 dim
            nn.ReLU(),
            # V

            nn.Linear(hidden_dim, hidden_dim),
            # hidden layer, 64 dim
            nn.ReLU(),
            # V

            nn.Linear(hidden_dim, output_dim)
            # V (taken into SolnNet)
        )
    
    def forward(self, ode_params):
        return self.net(ode_params)

##### Solution Network
Given trained weights from Hypernetwork, makes a predection with a single layer neural network.

In [7]:
class SolutionNet(nn.Module):
    def forward(self, x, meta_params):

        # pytorch tensor operations-
        
        w1 = meta_params[:, :16].view(-1, 4, 4)
        # 

        b1 = meta_params[:, 16:20].view(-1, 4)    # Hidden layer bias
        w2 = meta_params[:, 20:24].view(-1, 4, 1) # Output weights
        b2 = meta_params[:, 24:25].view(-1, 1)    # Output bias
    
        x = x.unsqueeze(-1)  # [batch_size, num_points, 1]
        h = torch.relu(torch.matmul(x, w1) + b1.unsqueeze(1))
        return torch.matmul(h, w2) + b2.unsqueeze(1)
